## Regression With NN - Boston Housing Prices

### Data Exploration

In [2]:
from keras.datasets import boston_housing

(boston_train_data, boston_train_targets), (boston_test_data, boston_test_targets) = boston_housing.load_data(test_split=0.2,seed=113)

# test_split: fraction of the data to reserve as test set.

In [3]:
print(f"dim/rank/ no of axes of train_data >> {boston_train_data.ndim}")
print(f"shape of train_data>> {boston_train_data.shape}")
print(f"shape of test_data>> {boston_test_data.shape}")

print('\n')
print(f"dim/rank/ no of axes of train_labels >> {boston_train_targets.ndim}")
print(f"shape of train_targets>> {boston_train_targets.shape}")
print(f"shape of test_targets>> {boston_test_targets.shape}")

print('\n')
print(f"Type of train_data first element >> {type(boston_train_data[0])}")
print(f"Size of train_data first element >> {(len(boston_train_data[0]))}")
print(f"First Record train_data>> {boston_train_data[0,]}")

print('\n')
print(f"Type of train_targets >> {type(boston_train_targets)}")
print(f"Type of train_targets first element >> {type(boston_train_targets[0])}")
print(f"Size of train_targets>> {(len(boston_train_targets))}")
print(f"First  of train_targets>> {(boston_train_targets)[0]}")

dim/rank/ no of axes of train_data >> 2
shape of train_data>> (404, 13)
shape of test_data>> (102, 13)


dim/rank/ no of axes of train_labels >> 1
shape of train_targets>> (404,)
shape of test_targets>> (102,)


Type of train_data first element >> <class 'numpy.ndarray'>
Size of train_data first element >> 13
First Record train_data>> [  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


Type of train_targets >> <class 'numpy.ndarray'>
Type of train_targets first element >> <class 'numpy.float64'>
Size of train_targets>> 404
First  of train_targets>> 15.2


#### Content Exploration - Pandas

In [4]:
import pandas as pd
print(" \nTraining Data >> ")
print (pd.DataFrame(boston_train_data).head())
print(" \nTraining Target >> ")
print (pd.DataFrame(boston_train_targets).head())

 
Training Data >> 
        0     1      2    3      4      5      6       7     8      9     10  \
0  1.23247   0.0   8.14  0.0  0.538  6.142   91.7  3.9769   4.0  307.0  21.0   
1  0.02177  82.5   2.03  0.0  0.415  7.610   15.7  6.2700   2.0  348.0  14.7   
2  4.89822   0.0  18.10  0.0  0.631  4.970  100.0  1.3325  24.0  666.0  20.2   
3  0.03961   0.0   5.19  0.0  0.515  6.037   34.5  5.9853   5.0  224.0  20.2   
4  3.69311   0.0  18.10  0.0  0.713  6.376   88.4  2.5671  24.0  666.0  20.2   

       11     12  
0  396.90  18.72  
1  395.38   3.11  
2  375.52   3.26  
3  396.90   8.01  
4  391.43  14.65  
 
Training Target >> 
      0
0  15.2
1  42.3
2  50.0
3  21.1
4  17.7


### Data Preprocessing

#### Normalisation 

Since the scale of the data varies significantly from one feature to the other, they need to be  normalised.

In [5]:
import numpy as np
print(boston_train_data.mean(axis=0))

boston_mean = boston_train_data.mean(axis=0)
boston_std = boston_train_data.std(axis=0)
boston_normed_train_data = (boston_train_data - boston_mean)/ boston_std

# To normalise the test data, mean/std should be used from the train data & not the test data
boston_normed_test_data = (boston_test_data - boston_mean)/ boston_std

import pandas as pd
print(" \nTraining Data >> ")
print (pd.DataFrame(boston_train_data).head())

print(" \nNormalised Training Data >> ")
print (pd.DataFrame(boston_normed_train_data).head())

print(" \nNormalised Test Data >> ")
print (pd.DataFrame(boston_normed_test_data).head())

[3.74511057e+00 1.14801980e+01 1.11044307e+01 6.18811881e-02
 5.57355941e-01 6.26708168e+00 6.90106436e+01 3.74027079e+00
 9.44059406e+00 4.05898515e+02 1.84759901e+01 3.54783168e+02
 1.27408168e+01]
 
Training Data >> 
        0     1      2    3      4      5      6       7     8      9     10  \
0  1.23247   0.0   8.14  0.0  0.538  6.142   91.7  3.9769   4.0  307.0  21.0   
1  0.02177  82.5   2.03  0.0  0.415  7.610   15.7  6.2700   2.0  348.0  14.7   
2  4.89822   0.0  18.10  0.0  0.631  4.970  100.0  1.3325  24.0  666.0  20.2   
3  0.03961   0.0   5.19  0.0  0.515  6.037   34.5  5.9853   5.0  224.0  20.2   
4  3.69311   0.0  18.10  0.0  0.713  6.376   88.4  2.5671  24.0  666.0  20.2   

       11     12  
0  396.90  18.72  
1  395.38   3.11  
2  375.52   3.26  
3  396.90   8.01  
4  391.43  14.65  
 
Normalised Training Data >> 
         0         1         2         3         4         5         6   \
0 -0.272246 -0.483615 -0.435762 -0.256833 -0.165227 -0.176443  0.813062   
1 -0

### Training

#### Network

In [6]:
from keras import models
from keras import layers

def build_regression_network():
    boston_model = models.Sequential()
    boston_model.add(layers.Dense(64, activation='relu', input_shape=(13,)))
    boston_model.add(layers.Dense(64, activation='relu'))
    boston_model.add(layers.Dense(1))


    boston_model.compile(optimizer='rmsprop',
                  loss='mse',
                  metrics=['mae'])

    return boston_model

#### K-fold Cross Validation & Evaluation

In [7]:
import numpy as np
k= 2
assert(len(boston_normed_train_data)%k == 0)
print(f"shape of train_data>> {boston_normed_train_data.shape}")
boston_fold_len = int(len(boston_normed_train_data)/k)
print(f"No of records in Train>> {boston_fold_len}  ")

boston_all_mae_histories = []
for i in range(k):
    boston_fold_val_data = boston_normed_train_data[i*boston_fold_len:((i+1)*boston_fold_len)]   
    boston_fold_val_targets = boston_train_targets[i*boston_fold_len:((i+1)*boston_fold_len)]   
    boston_fold_train_data = np.concatenate((boston_normed_train_data[:i*boston_fold_len],
                   boston_normed_train_data[((i+1)*boston_fold_len):]),axis=0)
    boston_fold_train_targets = np.concatenate((boston_train_targets[:i*boston_fold_len],
                   boston_train_targets[((i+1)*boston_fold_len):]),axis=0)
    print(f""""Processing fold: {i}, shape of:
          train_data >>    {boston_fold_train_data.shape},
          train_targets >> {boston_fold_train_targets.shape},
          val_data >>      {boston_fold_val_data.shape},
          val_targets >>   {boston_fold_val_targets.shape}""")
    
    boston_model = build_regression_network()
    
    boston_history_item = boston_model.fit(x=boston_fold_train_data,
                        y=boston_fold_train_targets,
                        batch_size=5,
                        epochs=20,
                        verbose=0,
                        validation_data=(boston_fold_val_data,boston_fold_val_targets))
    
    print(f"History Keys >> {boston_history_item.history.keys()}")
    print(f"History Val MAE >> {boston_history_item.history['val_mean_absolute_error']}")
    
    boston_mae_history = boston_history_item.history['val_mean_absolute_error']
    boston_all_mae_histories.append(boston_mae_history)

print(f"\nThe MAE for each fold for each iteration >>> {boston_all_mae_histories}")

print(f"\nThe Average MAE >>> {np.mean(boston_all_mae_histories)}")

shape of train_data>> (404, 13)
No of records in Train>> 202  
"Processing fold: 0, shape of:
          train_data >>    (202, 13),
          train_targets >> (202,),
          val_data >>      (202, 13),
          val_targets >>   (202,)
History Keys >> dict_keys(['val_loss', 'val_mean_absolute_error', 'loss', 'mean_absolute_error'])
History Val MAE >> [16.132320411134476, 8.777061169690425, 5.150761212452804, 3.981820741502365, 3.28142247813763, 3.159424508857255, 2.950886081350912, 2.9014931114593354, 2.9258777345761215, 2.813578717779405, 2.917053371667862, 2.7161632746753126, 2.816344657156727, 2.697339021628446, 2.502317920474723, 2.84482609812576, 2.4165265046724, 2.418603831588632, 2.4610011793009123, 2.4182803264938957]
"Processing fold: 1, shape of:
          train_data >>    (202, 13),
          train_targets >> (202,),
          val_data >>      (202, 13),
          val_targets >>   (202,)
History Keys >> dict_keys(['val_loss', 'val_mean_absolute_error', 'loss', 'mean_absol